# <u>Chapter 9</u>: Generating Text in Chatbots

In [8]:
import sys
import subprocess
import pkg_resources

# Find out which packages are missing.
installed_packages = {dist.key for dist in pkg_resources.working_set}
required_packages = {'torch', 'transformers', 'gradio'}
missing_packages = required_packages - installed_packages

# If there are missing packages install them.
if missing_packages:
    print('Installing the following packages: ' + str(missing_packages))
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing_packages], stdout=subprocess.DEVNULL)

## Creating a generative chatbot

We implement a generative chatbot and to make the interaction more enjoyable, we use the pre-trained model _microsoft/DialoGPT-medium_ that is specifically designed for this task. 

The _chat_ method that follows is responsible for receiving the user input and generating a response from the bot.

In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
 
# Load the models.
model_name = "microsoft/DialoGPT-medium"
 
gpt2_tokenizer = AutoTokenizer.from_pretrained(model_name)
gpt2_model = AutoModelForCausalLM.from_pretrained(model_name)

# Chat with the bot using a new input and the previous history.
def chat(input, history=[], gen_kwargs=[]):
    
    # Tokenize the input.
    input_ids = gpt2_tokenizer.encode(input+gpt2_tokenizer.eos_token, return_tensors='pt')

    # Update the dialogue history.
    bot_input_ids = torch.cat([torch.LongTensor(history), input_ids], dim=-1)

    # Generate the response of the bot. 
    new_history = gpt2_model.generate(bot_input_ids, **gen_kwargs).tolist()

    # Convert the tokens to text.
    output = gpt2_tokenizer.decode(new_history[0]).split("<|endoftext|>")
    output = [(output[i], output[i+1]) for i in range(0, len(output)-1, 2)]

    return output, new_history

We simulate a multi-turn dialog generation requesting advice from the chatbot.

In [10]:
# Parameters for the model.
gen_kwargs = {
    "max_length":1000,
    "min_length":-1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": False,
    "pad_token_id": gpt2_tokenizer.eos_token_id
}

# Simulate the chat.
me = ["What is your best advice?", "Does money buy happiness?", "Do you have money?", "Did you buy happiness?", "Well done..."]
history = []

for user_input in me:
    output, history = chat(user_input, history, gen_kwargs)
    print("Me:\t", user_input) 
    print("Bot:\t", output[len(output)-1][1])
    print("---------------") 

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Me:	 What is your best advice?
Bot:	 Don't be a loser.
---------------


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Me:	 Does money buy happiness?
Bot:	 It does if you're a loser.
---------------


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Me:	 Do you have money?
Bot:	 I have a lot of money.
---------------


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Me:	 Did you buy happiness?
Bot:	 I bought happiness.
---------------
Me:	 Well done...
Bot:	 I'm a happy guy.
---------------


When the _do_sample_ parameter is set to _False_ greedy decoding is used. Otherwise sampling is used. 

In [11]:
# Parameters for the model.
gen_kwargs = {
    "max_length":1000,
    "min_length":-1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": gpt2_tokenizer.eos_token_id
}

# Simulate the chat.
history = []

for user_input in me:
    output, history = chat(user_input, history, gen_kwargs)
    print("Me:\t", user_input) 
    print("Bot:\t", output[len(output)-1][1])
    print("---------------") 

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Me:	 What is your best advice?
Bot:	 Get a head and just never shed it.
---------------


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Me:	 Does money buy happiness?
Bot:	 Ride a bike. Price is right
---------------


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Me:	 Do you have money?
Bot:	 No. Yes.
---------------


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Me:	 Did you buy happiness?
Bot:	 No. No I didn't. Congrats.
---------------
Me:	 Well done...
Bot:	 Thank you. It's no problem.
---------------


## Creating the GUI.

In the following code snippet, we create the visual components of the interface and show the GUI.

In [12]:
import textwrap
import datetime as dt
from tkinter import *

# Parameters for the model.
gen_kwargs = {
    "max_length":1000,
    "min_length":-1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": False,
    "pad_token_id": gpt2_tokenizer.eos_token_id
}

# A GUI for the chat application.
class Chatty:

    # Keep track of the dialogue history.
    history = []
    
    def __init__(self):
        self.window = Tk()
        self.setup_gui()
        
    def run(self):
        self.window.mainloop()
    
    # Method to create the different GUI elements.
    def setup_gui(self):
        self.window.title("chatty")
        self.window.resizable(width=False, height=False)
        self.window.configure(width=550, height=500, bg="#17202A")
        
        # Add the head label.
        self.profile_img = PhotoImage(file='./images/user-online.png')
        head_label = Label(self.window, bg="#17202A", fg="#EAECEE", compound=LEFT,
                           text="Nick Brioche", font="Helvetica 13 bold", pady=0, image=self.profile_img)
        head_label.place(relwidth=1)
        
        # Add a line divider.
        div_line = Label(self.window, width=450, bg="#ABB2B9")
        div_line.place(relwidth=1, rely=0.07, relheight=0.012)
        
        # Add the text area.
        self.text_area = Text(self.window, width=20, height=2, bg="white", fg="#EAECEE",
                                font="Helvetica 14", padx=15, pady=5)
        self.text_area.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_area.configure(cursor="arrow", state=DISABLED)
        
        # Add a scroll bar.
        scroll_bar = Scrollbar(self.text_area)
        scroll_bar.place(relheight=1, relx=1.0)
        scroll_bar.configure(command=self.text_area.yview)
        
        # Add a bottom label.
        bottom_label = Label(self.window, bg="#ABB2B9", height=80)
        bottom_label.place(relwidth=1, rely=0.825)
        
        # Add an input entry box.
        self.input_entry = Entry(bottom_label, bg="#6a747e", fg="#EAECEE", font="Helvetica 14")
        self.input_entry.place(relwidth=0.74, relheight=0.07, rely=0.0, relx=0.0)
        self.input_entry.focus()
        self.input_entry.bind("<Return>", self.on_enter_pressed)
        
        # Create the send button.
        send_button = Button(bottom_label, text="Send", bd=0, bg="#ABB2B9", 
                             command=lambda: self.on_enter_pressed(None))
        self.send_img = PhotoImage(file='./images/send.png')
        send_button.config(image=self.send_img)
        send_button.place(relx=0.77, rely=0.0, relheight=0.07, relwidth=0.22)
    
    # Method to capture the press of the Enter button.
    def on_enter_pressed(self, event):
        msg = self.input_entry.get()
        self.chatbot(msg)

    # Chat with the bot.
    def chatbot(self, msg):

        # Skip empty input.
        if not msg: return
        
        # Show the request on the GUI.
        self.input_entry.delete(0, END)
        request = f"{msg}\n\n"
        self.text_area.configure(state=NORMAL)
        self.text_area.tag_config('request', foreground="black", wrap='word')
        self.text_area.insert(END, request, 'request')
        self.text_area.configure(state=DISABLED)
        
        # Get the bot's response.
        output, self.history = chat(msg, self.history, gen_kwargs)
        response = f"{output[len(output)-1][1]}\n\n"
       
        # Show the response on the GUI.
        self.text_area.configure(state=NORMAL)
        self.text_area.tag_config('response', justify='right', foreground="black", 
                                    background="lightgreen", wrap='word', rmargin=10)
        self.text_area.insert(END, response, 'response')
        self.text_area.configure(state=DISABLED)
        
        self.text_area.see(END)

# Uncomment to show the GUI when running the notebook in your local PC.       
# # Run the chat application.
# if __name__ == "__main__":
#     app = Chatty()
#     app.run()

### Web version

`Gradio` (https://gradio.app/) can be used to create the web version of the GUI, so we need to adapt the Python code to include HTML tags.

In [13]:
# Chat with the bot using a new input and the previous history.
# Return a basic HTML including the dialogue.
def chat_html(input, history=[]):

    # Skip empty input.
    if not input: return

    output, history = chat(input, history, gen_kwargs)
    
    # Create the HTML.
    html = "<div class='chatbot'>"
    for tuple in output:
        for m, msg in enumerate(tuple):
            cls = "user" if m%2 == 0 else "bot"
            html += "<div class='msg {}'> {}</div>".format(cls, msg)
        html += "</div>"
    
    return html, history

Using the specifics of the gradio module, we can run the chat application on a local web server.

In [15]:
import gradio as gr

# Define the chat function (replace this with your actual chat logic)
def chat_html(message, state):
    # Your chat logic goes here
    response = "Bot: Hello! I'm a simple chatbot."
    return f"<div class='msg bot'>{response}</div>", state

# Setup the style of the GUI.
css = """
.chatbox {display:flex;flex-direction:column}
.msg {padding:4px;margin-bottom:4px;border-radius:4px;width:80%}
.msg.user {background-color:cornflowerblue;color:white}
.msg.bot {background-color:lightgray;align-self:self-end}
.footer {display:none !important}
"""

# Launch the interface.
gr.Interface(fn=chat_html, theme="default",
             inputs=[gr.components.Textbox(placeholder="Type a message..."), "state"],
             outputs=["html", "state"], css=css).launch()


IMPORTANT: You are using gradio version 3.3, however version 3.14.0 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x1a23e6dc3d0>, 'http://127.0.0.1:7863/', None)

## What we have learned …

| |
| --- |
| **Tools**<ul><li>GUI Programming</li></ul> |
| |

## Author Information

- **Author:** Nikos Tsourakis
- **Email:** nikos@tsourakis.net
- **Website:** [tsourakis.net](https://tsourakis.net)
- **Date:** November 20, 2023